In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
import re

In [47]:
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        try:
            nombre = soup.find("h1", class_="ad-title")
            nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        except:
            nombre = "Error al acceder al inmueble"

        try:
            agencia = soup.find("p", class_="owner-info__name")
            agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"
        except:
            agencia = "Agencia no disponible"

        try:
            features_summary = soup.find("ul", class_="features-summary")
            precio_m2 = None
            if features_summary:
                items = features_summary.find_all("li", class_="features-summary__item")
                for item in items:
                    if "€/m²" in item.text:
                        precio_m2 = item.text.strip()
                        break
            precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        except:
            precio_m2 = "Error al procesar precio m²"
        
        try:
            precio = soup.find("div", class_="price__value jsPriceValue")
            precio = precio.text.strip() if precio else "Error al procesar precio"
        except:
            precio = "Error al procesar precio"
        
        try:
            superficie = soup.find("span", class_="features__value")
            superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        except:
            superficie = "Error al procesar superficie"
        
        try:
            actualizacion = soup.find("div", class_="details__block last-update")
            actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        except:
            actualizacion = "Error al procesar actualizacion"

        try:
            consumo = soup.find_all("span", class_="energy-certificate__tag")
            consumo_etiquetas = []
            for etiqueta in consumo:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                    if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                        consumo_etiquetas.append(letra_consumo.upper())
            consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"
        except:
            consumo = "Error al procesar consumo"

        try:
            emisiones = soup.find_all("span", class_="energy-certificate__tag")
            emisiones_etiquetas = []
            for etiqueta in emisiones:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                    if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                        emisiones_etiquetas.append(letra_emisiones.upper())
            emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"
        except:
            emisiones = "Error al procesar emisiones"

        try:
            caracteristicas = soup.find("div", class_="features-container")
            detalles = {}
            if caracteristicas:
                secciones = caracteristicas.find_all("div", class_="features__content")
                for seccion in secciones:
                    features = seccion.find_all("div", class_="features__feature")
                    for feature in features:
                        label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                        value = feature.find("span", class_="features__value").text.strip() if feature.find("span", class_="features__value") else "Si"
                        detalles[label] = value
        except:
            detalles = {}

        # Aquí vamos a extraer el Código Postal e Identificador mediante la URL de cada inmueble
        try:
            codigo_postal_match = re.search(r'(\d{5})-', url_inmueble)
            codigo_postal = codigo_postal_match.group(1) if codigo_postal_match else 'NaN'
        except Exception as e:
            codigo_postal = f"Error al extraer código postal: {e}"
        
        try:
            identificador_match = re.search(r'-(\d+_\d+)', url_inmueble)
            identificador = identificador_match.group(1) if identificador_match else 'NaN'
        except Exception as e:
            identificador = f"Error al extraer identificador: {e}"
        
        # Aquí extraemos el timestamp de cada extracción de datos, que nos será util a la hora de actualizar la db
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Aquí extraemos las coordenadas (latitud y longitud)
        try:
            location_div = soup.find("div", class_="location")
            data_params = location_div["data-params"]
            
            lat_match = re.search(r'latitude=([-0-9.]+)', data_params)
            lon_match = re.search(r'longitude=([-0-9.]+)', data_params)
            
            latitud = lat_match.group(1) if lat_match else 'NaN'
            longitud = lon_match.group(1) if lon_match else 'NaN'
            
            coordenadas = f"{latitud}, {longitud}"
        except Exception as e:
            coordenadas = f"Error al extraer coordenadas: {e}"
            
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas
    except Exception as e:
        return ("Error al acceder al inmueble",) * 9 + ({}, "", "", datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "")

In [48]:
def obtener_inmuebles_paginas(base_url, paginas=2):
    todos_los_inmuebles = []
    columnas = set()

    for i in range(1, paginas + 1):
        print(f"Procesando nueva página")
        url_pagina = f"{base_url}{i}/"
        
        try:
            response = requests.get(url_pagina)
            soup = BeautifulSoup(response.text, "html.parser")
            
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles, codigo_postal, identificador, timestamp, coordenadas = obtener_detalles_inmueble(href)
                
                ubicacion = base_url.split('/')[-2].split('-')[-1]
                #corregido extraer ubicacion para compras al tener las urls y que no se quede con las paginas
                #ubicacion = url_pagina.split('/')[-2]
                
                inmueble = {
                    "nombre": nombre,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "consumo": consumo,
                    "emisiones": emisiones,
                    "ubicacion": ubicacion,
                    "codigo_postal": codigo_postal,
                    "identificador": identificador,
                    "timestamp": timestamp,
                    "coordenadas": coordenadas
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
                columnas.update(inmueble.keys())
            
        except Exception as e:
            print(f"Error al procesar la página {i}: {e}")
        
        time.sleep(2)
    
    if todos_los_inmuebles:
        return pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    else:
        return pd.DataFrame(columns=list(columnas))

In [49]:
#test con una url
urls_zonas_paginas = {
        #ANDALUCIA
    "https://www.pisos.com/alquiler/pisos-almeria_capital/": 2,
    "https://www.pisos.com/alquiler/pisos-roquetas_de_mar/": 2
    }

# Aquí creo una función que recorra cada una de las paginas de las URLs del diccionario 
def obtener_inmuebles_varias_zonas(urls_zonas_paginas):
    todos_los_inmuebles = []
    columnas = set()
    
    for base_url, paginas in urls_zonas_paginas.items():
        inmuebles_zona = obtener_inmuebles_paginas(base_url, paginas)
        todos_los_inmuebles.extend(inmuebles_zona.to_dict('records'))
        if not inmuebles_zona.empty:
            columnas.update(inmuebles_zona.columns)
    
    df_alquileres = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_alquileres

In [20]:
# Aquí vamos a hacer un diccionario con las URLs de las zonas de los inmuebles que queremos obtener, 
# con la cantidad de paginas correspondientes
urls_zonas_paginas = {
        #ANDALUCIA
    "https://www.pisos.com/alquiler/pisos-almeria_capital/": 7,
    "https://www.pisos.com/alquiler/pisos-roquetas_de_mar/": 4,
    "https://www.pisos.com/alquiler/pisos-vera/": 2,
    "https://www.pisos.com/alquiler/pisos-chiclana_de_la_frontera/": 5,
    "https://www.pisos.com/alquiler/pisos-sanlucar_de_barrameda/": 4,
    "https://www.pisos.com/alquiler/pisos-cadiz_capital/": 4,
    "https://www.pisos.com/alquiler/pisos-cordoba_capital_zona_urbana/": 3,
    "https://www.pisos.com/alquiler/pisos-area_de_granada_granada_capital/": 14,
    "https://www.pisos.com/alquiler/pisos-almunecar/": 5,
    "https://www.pisos.com/alquiler/pisos-isla_cristina/": 2,
    "https://www.pisos.com/alquiler/pisos-punta_umbria/": 2,
    "https://www.pisos.com/alquiler/pisos-jaen/": 3,
    "https://www.pisos.com/alquiler/pisos-marbella/": 14,
    "https://www.pisos.com/alquiler/pisos-malaga_capital_zona_urbana/": 9,
    "https://www.pisos.com/alquiler/pisos-estepona/": 4,
    "https://www.pisos.com/alquiler/pisos-sevilla_capital/": 7,
        #ARAGON
    "https://www.pisos.com/alquiler/pisos-huesca/": 2,
    "https://www.pisos.com/alquiler/pisos-teruel/": 1,
    "https://www.pisos.com/alquiler/pisos-zaragoza/": 5,
        #CANTABRIA
    "https://www.pisos.com/alquiler/pisos-santander/": 5,
    "https://www.pisos.com/alquiler/pisos-laredo/": 2,
        #CASTILLA Y LEON
    "https://www.pisos.com/alquiler/pisos-avila/": 1,
    "https://www.pisos.com/alquiler/pisos-burgos/": 2,
    "https://www.pisos.com/alquiler/pisos-leon/": 3,
    "https://www.pisos.com/alquiler/pisos-palencia/": 1,
    "https://www.pisos.com/alquiler/pisos-salamanca/": 10,
    "https://www.pisos.com/alquiler/pisos-soria/": 1,
    "https://www.pisos.com/alquiler/pisos-valladolid/": 5,
    "https://www.pisos.com/alquiler/pisos-zamora/": 1,
        #CASTILLA LA MANCHA
    "https://www.pisos.com/alquiler/pisos-albacete/": 2,
    "https://www.pisos.com/alquiler/pisos-ciudad_real/": 1,
    "https://www.pisos.com/alquiler/pisos-cuenca/": 2,
    "https://www.pisos.com/alquiler/pisos-guadalajara/": 1,
    "https://www.pisos.com/alquiler/pisos-toledo/": 3,
        #CATALUÑA
    "https://www.pisos.com/alquiler/pisos-barcelona_capital/": 27,
    "https://www.pisos.com/alquiler/pisos-sitges/": 2,
    "https://www.pisos.com/alquiler/pisos-girona/": 5,
    "https://www.pisos.com/alquiler/pisos-lleida/": 2,
    "https://www.pisos.com/alquiler/pisos-tarragona/": 5,
        #COMUNIDAD DE MADRID
    "https://www.pisos.com/alquiler/pisos-madrid_capital_zona_urbana/": 62,
    "https://www.pisos.com/alquiler/pisos-pozuelo_de_alarcon/": 2,
    "https://www.pisos.com/alquiler/pisos-madrid_norte_la_moraleja/": 2,
        #COMUNIDAD VALENCIANA
    "https://www.pisos.com/alquiler/pisos-alicante/": 51,
    "https://www.pisos.com/alquiler/pisos-castellon_castello/": 8,
    "https://www.pisos.com/alquiler/pisos-valencia/": 45,
        #EXTREMADURA
    "https://www.pisos.com/alquiler/pisos-badajoz/": 4,
    "https://www.pisos.com/alquiler/pisos-caceres/": 2,
        #GALICIA
    "https://www.pisos.com/alquiler/pisos-a_coruna/": 8,
    "https://www.pisos.com/alquiler/pisos-lugo/": 1,
    "https://www.pisos.com/alquiler/pisos-ourense/": 2,
    "https://www.pisos.com/alquiler/pisos-pontevedra/": 9,
        #ISLAS BALEARES
    "https://www.pisos.com/alquiler/pisos-islas_baleares_illes_balears/": 13,
        #ISLAS CANARIAS
    "https://www.pisos.com/alquiler/pisos-las_palmas/": 9,
    "https://www.pisos.com/alquiler/pisos-santa_cruz_de_tenerife/": 7,
        #LA RIOJA
    "https://www.pisos.com/alquiler/pisos-la_rioja/": 1,
        #NAVARRA
    "https://www.pisos.com/alquiler/pisos-navarra_nafarroa/": 3,
        #PAIS VASCO
    "https://www.pisos.com/alquiler/pisos-alava_araba/": 1,
    "https://www.pisos.com/alquiler/pisos-guipuzcoa_gipuzkoa/": 2,
    "https://www.pisos.com/alquiler/pisos-vizcaya_bizkaia/": 10,
        #ASTURIAS
    "https://www.pisos.com/alquiler/pisos-oviedo/": 5,
    "https://www.pisos.com/alquiler/pisos-gijon_concejo_xixon_conceyu_gijon/": 3,
        #REGION DE MURCIA
    "https://www.pisos.com/alquiler/pisos-murcia_capital/": 6,
    "https://www.pisos.com/alquiler/pisos-cartagena/": 4,
    "https://www.pisos.com/alquiler/pisos-la_manga_del_mar_menor/": 2
}

# Aquí creo una función que recorra cada una de las paginas de las URLs del diccionario 
def obtener_inmuebles_varias_zonas(urls_zonas_paginas):
    todos_los_inmuebles = []
    columnas = set()
    
    for base_url, paginas in urls_zonas_paginas.items():
        inmuebles_zona = obtener_inmuebles_paginas(base_url, paginas)
        todos_los_inmuebles.extend(inmuebles_zona.to_dict('records'))
        if not inmuebles_zona.empty:
            columnas.update(inmuebles_zona.columns)
    
    df_alquileres = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_alquileres

In [50]:
# Aquí se procesa cada pagina para rellenar el df de alquileres
df_alquileres = obtener_inmuebles_varias_zonas(urls_zonas_paginas)

Procesando nueva página
Procesando nueva página
Procesando nueva página
Procesando nueva página


In [51]:
df_alquileres

,Armarios empotrados,Interior,Exterior,Balcón,Superficie útil,Amueblado,Calefacción,Orientación,precio,coordenadas,...,Piscina,Trastero,Luz,Portero automático,Adaptado a personas con movilidad reducida,Calle alumbrada,nombre,Antigüedad,emisiones,precio_m2
0,NaN,NaN,Si,Si,90 m²,NaN,NaN,NaN,700 €/mes,"36.8378928, -2.4597868",...,NaN,NaN,NaN,NaN,NaN,NaN,Piso en Avenida de la Estación,Más de 50 años,Error al procesar emisiones,6 €/m²
1,NaN,NaN,Si,Si,90 m²,NaN,NaN,NaN,700 €/mes,"36.8378928, -2.4597868",...,NaN,NaN,NaN,NaN,NaN,NaN,Piso en Avenida de la Estación,Más de 50 años,Error al procesar emisiones,6 €/m²
2,Sin especificar,NaN,NaN,NaN,120 m²,Amueblado,NaN,NaN,1.000 €/mes,"36.8419326, -2.4539274",...,NaN,NaN,NaN,Si,Si,NaN,Piso en Oliveros-Altamira-Barrio Alto,Entre 30 y 50 años,Error al procesar emisiones,8 €/m²
3,1,NaN,NaN,NaN,63 m²,NaN,Si,Este,740 €/mes,"36.8543269, -2.4431899",...,NaN,Si,NaN,NaN,NaN,NaN,Piso en Avda. Mediterraneo,Entre 20 y 30 años,"E, D",10 €/m²
4,NaN,NaN,NaN,NaN,73 m²,Si,NaN,NaN,800 €/mes,"36.8317798, -2.4501068",...,NaN,NaN,NaN,NaN,NaN,NaN,Piso en calle Vera,NaN,Error al procesar emisiones,10 €/m²
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Sin especificar,NaN,Si,NaN,99 m²,Si,NaN,NaN,1.100 €/mes,"36.8125316, -2.5722794",...,Sin especificar,NaN,NaN,Si,NaN,NaN,Piso en Aguadulce Norte,Entre 5 y 10 años,"E, E",11 €/m²
118,2,NaN,NaN,NaN,70 m²,Si,NaN,NaN,700 €/mes,"36.724553, -2.6262374",...,Comunitaria,NaN,NaN,NaN,NaN,NaN,Piso en Urbanización de Roquetas-Las Marinas,NaN,"E, E",8 €/m²
119,1,NaN,NaN,NaN,57 m²,Si,NaN,NaN,500 €/mes,"36.7388209, -2.6183405",...,NaN,NaN,NaN,NaN,NaN,NaN,Piso en Urbanización de Roquetas-Las Marinas,NaN,"E, E",8 €/m²
120,1,NaN,NaN,NaN,75 m²,Si,Si,Oeste,850 €/mes,"36.8019103, -2.5804847",...,Comunitaria,NaN,NaN,NaN,NaN,NaN,Piso en calle Canterbury,Entre 20 y 30 años,Error al procesar emisiones,10 €/m²


In [23]:
df_alquileres.shape

(12598, 61)

In [24]:
df_alquileres.columns

Index(['Luz', 'Jardín', 'Sistema de seguridad', 'Conservación',
       'Vidrios dobles', 'Se aceptan mascotas', 'timestamp',
       'Carpintería interior', 'Calefacción', 'Trastero', 'href',
       'Calle asfaltada', 'Tipo de casa', 'Cocina equipada', 'Agua',
       'emisiones', 'Aire acondicionado', 'Terraza', 'Armarios empotrados',
       'actualizacion', 'superficie', 'Balcón', 'Gastos de comunidad',
       'Urbanizado', 'Superficie construida', 'Garaje', 'consumo',
       'Puerta blindada', 'Adaptado a personas con movilidad reducida',
       'identificador', 'Ascensor', 'Superficie útil', 'nombre',
       'Alcantarillado', 'Planta', 'Baños', 'Portero automático', 'Exterior',
       'Soleado', 'Piscina', 'Calle alumbrada', 'precio_m2', 'precio',
       'Lavadero', 'Orientación', 'Amueblado', 'Interior', 'Habitaciones',
       'Gas', 'Tipo suelo', 'Carpintería exterior', 'Superficie solar',
       'Comedor', 'Antigüedad', 'codigo_postal', 'ubicacion', 'agencia',
       'Chimenea', '

In [28]:
#Guardamos el df resultante en un .csv
df_alquileres.to_csv('alquileres_scrap_completo.csv', index=False)

In [29]:
df = pd.read_csv('alquileres_scrap_completo.csv')

In [30]:
df["ubicacion"].value_counts()

ubicacion
madrid_capital_zona_urbana         1864
alicante                           1497
valencia                           1337
barcelona_capital                   811
area_de_granada_granada_capital     408
                                   ... 
avila                                14
alava_araba                           7
soria                                 5
teruel                                5
zamora                                4
Name: count, Length: 64, dtype: int64